In [50]:
import pandas as pd
import numpy as np
import sqlite3 as db
from io import StringIO
from csv import writer 

In [3]:
con = db.connect('accounts.db')

In [61]:
# gather the total amount of claims that occurred
sql = 'SELECT COUNT(*) FROM claim_event'
total_claim_events = pd.read_sql(sql, con)

In [62]:
# filter rows on claim_event on which sender is present on decay_amount table and total_lost > 0
sql = 'SELECT * FROM claim_event WHERE sender IN (SELECT sender FROM decay_amount WHERE total_lost > 0)'
claim_events = pd.read_sql(sql, con)
claim_events

,id,sender,height,amount,claim_action
0,63,evmos1s846s02zda3z6en6mta0guygwjqlhm4yaylkn7,87703,55957203930434002944aevmos,ACTION_DELEGATE
1,70,evmos1pkdhcxw0rf26h4z9sc682f63sszhv9tv4tc7jf,87703,15502803176750492160aevmos,ACTION_VOTE
2,71,evmos1z8hc706hklp2x9ayyzz5emxa32y0djmyc3hnnx,87703,15912896986481662976aevmos,ACTION_EVM
3,73,evmos15wj8ztxpurjxtysp68quky07dca4karrhmncyl,87703,3451904960222769664aevmos,ACTION_EVM
4,92,evmos168d84slcyqfypnck0pkj5qu95u75q0pevj96ss,87704,1762581660408326912aevmos,ACTION_DELEGATE
...,...,...,...,...,...
52274,520019,evmos1ejf5757z2nnxqynrxx64x8h5exjys8l9l78h25,5063003,6988309457762567aevmos,ACTION_DELEGATE
52275,520027,evmos19vra3y4cg8g7aymfv2gu242h4kcuytl2fufu25,5071272,11613620903356716aevmos,ACTION_IBC_TRANSFER
52276,520033,evmos1ama6mf2nn7dw9jruz9gumvs9wwvrsv6z4pcc9h,5071523,18092508781755638aevmos,ACTION_IBC_TRANSFER
52277,520042,evmos1a66kkjer33wyzhpy7dw839ltk78f4ytkp3upm4,5072234,1223861626876166aevmos,ACTION_EVM


In [63]:
# from results of previous query, find the highest "height"
sql = 'SELECT *, MAX(height) FROM claim_event WHERE sender IN (SELECT sender FROM decay_amount WHERE total_lost > 0)'
max_height = pd.read_sql(sql, con)

In [64]:
# do the same but for min height
sql = 'SELECT *, MIN(height) FROM claim_event WHERE sender IN (SELECT sender FROM decay_amount WHERE total_lost > 0)'
min_height = pd.read_sql(sql, con)
min_height

,id,sender,height,amount,claim_action,MIN(height)
0,8595,evmos18jggakl5f6wjnavkwa78k06thjq3ff8httnaxs,87223,12046997276210648064aevmos,ACTION_DELEGATE,87223


In [65]:
# All decay amount
sql = 'SELECT * FROM decay_amount'
tda = pd.read_sql(sql, con)

# clean empty data
tda['initial_claimable_amount']=tda.initial_claimable_amount.replace('',np.nan).astype(float)
tda['vote_action']=tda.vote_action.replace('',np.nan).astype(float)
tda['ibc_action']=tda.ibc_action.replace('',np.nan).astype(float)
tda['delegate_action']=tda.delegate_action.replace('',np.nan).astype(float)
tda['evm_action']=tda.evm_action.replace('',np.nan).astype(float)
tda['total_lost']=tda.total_lost.replace('',np.nan).astype(float)
tda['total_claimed']=tda.total_claimed.replace('',np.nan).astype(float)

tda.count()

id                          168662
sender                      168662
vote_action                 122772
ibc_action                  106613
delegate_action             120183
evm_action                  132062
total_claimed               168662
total_lost                  155341
initial_claimable_amount    155341
total_lost_evmos            168662
dtype: int64

In [66]:
tda.sample(10)

,id,sender,vote_action,ibc_action,delegate_action,evm_action,total_claimed,total_lost,initial_claimable_amount,total_lost_evmos
33067,33068,evmos15nn8kamqhwhxg4v5nvag7q7mhv8rxxxy2hwycr,6.744260e+18,6.744260e+18,6.744260e+18,NaN,2.023278e+19,0.000000e+00,2.697704e+19,0.000000
161637,161638,evmos1grprc00frgjn9kqj58srqnvf660tvv3xtljl9h,1.762582e+18,1.762582e+18,1.762582e+18,1.762582e+18,7.050327e+18,0.000000e+00,7.050327e+18,0.000000
16374,16375,evmos1tklszf3qnv95lmug8l4mtj57rry0fxge7p2kz3,NaN,NaN,NaN,4.588250e+19,4.588250e+19,0.000000e+00,1.835300e+20,0.000000
109358,109359,evmos1zxg9va42y0h2wp99g7wzljj6t0qejz73xm9h4f,1.655404e+18,1.655246e+18,1.655386e+18,1.655195e+18,6.621232e+18,1.073871e+17,7.050327e+18,0.107387
152628,152629,evmos1ja6dvuucna8ttf92sczuadrxf467cy5fd8cfv9,NaN,2.942195e+18,2.942423e+18,2.942689e+18,8.827308e+18,2.104829e+19,9.596196e+19,21.048294
118250,118251,evmos1q6qgw03vp0puthe0e7dq2qh3kpwyx66x84zjjc,NaN,NaN,NaN,2.797860e+19,2.797860e+19,0.000000e+00,1.119144e+20,0.000000
74005,74006,evmos140dpqemj3nnex2e5kvxdzfqjpg8ka77k35a6e6,1.227520e+19,1.227520e+19,1.227520e+19,1.227520e+19,4.910080e+19,0.000000e+00,4.910080e+19,0.000000
6976,6977,evmos15du4acep2fv0yge5n752q70wmk93wn5n0c9v46,9.042448e+18,NaN,9.042448e+18,9.042448e+18,2.712734e+19,0.000000e+00,3.616979e+19,0.000000
150735,150736,evmos1kgja607m3krud6fmx7d834h8e2fshkdnjlxcps,NaN,8.870040e+19,NaN,NaN,8.870040e+19,NaN,NaN,0.000000
120071,120072,evmos10pdwxz9v2rr7pm76pq8ajz0zw7nfkhk3e3l65p,2.640712e+19,2.640712e+19,2.640712e+19,2.640712e+19,1.056285e+20,0.000000e+00,1.056285e+20,0.000000


In [67]:
# filter accounts that do not have `initial_claimable_amount` 
# and `total_lost_evmos` is less than 0, which means they got more in certain events than they should had.
tda = tda.loc[(tda['total_lost_evmos'] < 0) | (tda['initial_claimable_amount'].isna())]

In [68]:
tda.count()

id                          15676
sender                      15676
vote_action                  2258
ibc_action                  13830
delegate_action              2317
evm_action                   1865
total_claimed               15676
total_lost                   2355
initial_claimable_amount     2355
total_lost_evmos            15676
dtype: int64

In [69]:
# take some random rows to make sure the results are correct
tda.sample(5)

,id,sender,vote_action,ibc_action,delegate_action,evm_action,total_claimed,total_lost,initial_claimable_amount,total_lost_evmos
134128,134129,evmos1f82g6k5x4q9gukjrx06z0dulscgef9f4cms5j5,NaN,1.717862e+20,NaN,NaN,1.717862e+20,NaN,NaN,0.000000
100536,100537,evmos1nn0j7sy9f2aca43klqhr6ldhyl47h59mdzsjpv,NaN,4.588250e+19,NaN,NaN,4.588250e+19,NaN,NaN,0.000000
112570,112571,evmos1pxly5yz3ku9tt9gy0nme6e9tp6pn505xuf2yra,NaN,8.785041e+19,NaN,NaN,8.785041e+19,NaN,NaN,0.000000
54908,54909,evmos136pmk4ajdr4mpk7p8ewhx6hfehrrtph45p2779,5.956459e+19,NaN,5.956459e+19,5.956459e+19,1.786938e+20,-5.595720e+19,1.442953e+19,-55.957204
89949,89950,evmos1fzmlvnpecs8uzcag8f4mpemc7syp9wseyqukhq,NaN,5.595720e+19,NaN,NaN,5.595720e+19,NaN,NaN,0.000000


In [70]:
# now that I have all the accounts that had either a merged or migrated event I need to figure out how much they lost.
# I need to figure out the amount to claim they had at the time they performed the claim. 
# lets find the events that were performed after merge or migration.

In [71]:
# IDEAD - Maybe by filtering it out the merged events I will keep only migration events, if I can find those txs on chain I would be able to know the recipient of those events and then be able to know the migrations. 

# mma are all the Merge and Migrated accounts that happened in history with their respective events
mma = pd.DataFrame(columns=['id', 'sender', 'initial_claimable_amount', 'vote_action', 'ibc_action', 'delegate_action', 'evm_action'])
for _, row in tda.iterrows():
    ndf = pd.DataFrame({
        'id': row['id'],
        'sender': row['sender'],
        'initial_claimable_amount': row['initial_claimable_amount'],
        'vote_action': [np.nan],
        'ibc_action': [np.nan],
        'delegate_action': [np.nan],
        'evm_action': [np.nan]
    })
    if pd.isna(row['initial_claimable_amount']):
        mma = pd.concat([mma, row[['id', 'sender', 'initial_claimable_amount', 'vote_action', 'ibc_action', 'delegate_action', 'evm_action']].to_frame().T], ignore_index=True)
    else:
        if row['vote_action'] > row['initial_claimable_amount'] / 2:
            ndf['vote_action'] = row['vote_action']
        elif row['ibc_action'] > row['initial_claimable_amount'] / 2:
            ndf['ibc_action'] = row['ibc_action']
        elif row['delegate_action'] > row['initial_claimable_amount'] / 2:
            ndf['delegate_action'] = row['delegate_action']
        elif row['evm_action'] > row['initial_claimable_amount'] / 2:
            ndf['evm_action'] = row['evm_action']
        mma = pd.concat([mma, ndf], ignore_index=True)
mma.head()

,id,sender,initial_claimable_amount,vote_action,ibc_action,delegate_action,evm_action
0,3,evmos1ep6qpcv2c0p3clkxap7repf0t7jgyu2jm87fga,NaN,NaN,45066046577831641088.0,NaN,NaN
1,8,evmos1ddy4a8m7yfvl9edx3declp84tgc2la7d72y6qd,NaN,NaN,45882504083542867968.0,NaN,NaN
2,12,evmos1j83ghq0n5rcwn3e6lk5a3rlm5zeevd4nd8jml4,NaN,NaN,55957203930434002944.0,NaN,NaN
3,18,evmos1u8lskt0fvljr6t5gtcsjmut3nnk8f9x4kcu92u,NaN,NaN,83935805895651016704.0,NaN,NaN
4,30,evmos1aeh63khvseksw8k629498k3mlechp8klcs5y92,NaN,NaN,27978601965217001472.0,NaN,NaN


In [72]:
mma.sample(20)

,id,sender,initial_claimable_amount,vote_action,ibc_action,delegate_action,evm_action
8120,87215,evmos1468pzf99kjvwsknqmuxszk6vr5f44tv6rp9ldd,NaN,NaN,41119848019493453824.0,NaN,NaN
8490,91156,evmos16z6rc707vsdl75qj3vsqnfzt9gm3nqs9tka337,NaN,NaN,45882504083542867968.0,NaN,NaN
13991,150399,evmos15x79ymtckzvflyqg2flh58dlt5v5tt04njy8g9,NaN,NaN,101336891886399815680.0,NaN,NaN
1928,20689,evmos16cuc8p7xfpvfk48ey85t98usw30ydh08ku90fl,NaN,NaN,45882504083542867968.0,NaN,NaN
5451,58116,evmos18t6zuygqw9dxu44tetdn7ltspxvl5e5f3f7mln,4.095304e+19,140056569656982437888.0,NaN,NaN,NaN
14205,152650,evmos1utafw08nlyp8wpns8xz2kvdjyr5n0t7n9mwaxq,NaN,NaN,45882504083542867968.0,NaN,NaN
3785,40850,evmos1dlnpn325676rtjgfz8syxvzvjh54jyzuz5s0nz,2.530354e+19,62283089607983955968.0,NaN,NaN,NaN
5452,58119,evmos1vpwgf28phys4mvtn4tc8vw40t46sdj8yl2mutp,NaN,NaN,45882504083542867968.0,NaN,NaN
1654,17763,evmos107crmmgp8xc94f4hee8ems3hmch2rqsvk5caty,NaN,NaN,157796911944410857472.0,NaN,NaN
4562,48910,evmos1ss5vaxa2j2c7wlhtrnyeh0c9zsvjd5q9xcjun8,NaN,NaN,33154952591790624768.0,NaN,NaN


In [4]:
# START CLCEAN

# All decay amount
sql = 'SELECT * FROM decay_amount'
decay_amount = pd.read_sql(sql, con)

# clean empty data
decay_amount['initial_claimable_amount']=decay_amount.initial_claimable_amount.replace('',np.nan).astype(float)
decay_amount['vote_action']=decay_amount.vote_action.replace('',np.nan).astype(float)
decay_amount['ibc_action']=decay_amount.ibc_action.replace('',np.nan).astype(float)
decay_amount['delegate_action']=decay_amount.delegate_action.replace('',np.nan).astype(float)
decay_amount['evm_action']=decay_amount.evm_action.replace('',np.nan).astype(float)
decay_amount['total_lost']=decay_amount.total_lost.replace('',np.nan).astype(float)
decay_amount['total_claimed']=decay_amount.total_claimed.replace('',np.nan).astype(float)

decay_amount.count()


id                          168662
sender                      168662
vote_action                 122772
ibc_action                  106613
delegate_action             120183
evm_action                  132062
total_claimed               168662
total_lost                  155341
initial_claimable_amount    155341
total_lost_evmos            168662
dtype: int64

In [8]:
# all claim event
sql = 'SELECT * FROM claim_event'
claim_event = pd.read_sql(sql, con)

claim_event['amount'] = claim_event['amount'].str.replace('aevmos', '').astype(float)
claim_event['height'] = claim_event['height'].astype(int)

claim_event.sample(5)

,id,sender,height,amount,claim_action
188743,188744,evmos1dpq0dsaak2sh4ytl66uyvmmeue0pgmk0eyqzwy,174584,3.525163e+18,ACTION_DELEGATE
315371,315372,evmos17aqt0rzrhc5dkcuymc4al7e23zlus3m0xkjz9l,528567,3.525163e+18,ACTION_IBC_TRANSFER
293657,293658,evmos1ysnf59naklcru54hy969h5ysgfmae2qg7gv9ca,466843,4.588250e+19,ACTION_IBC_TRANSFER
146065,146066,evmos1r0jlnhuc3sryqpuq657lzs7ph0ljnz9444al70,144575,2.988626e+19,ACTION_EVM
15263,15264,evmos1fk9jk7al9uqwdw3ltvpjutaruav7kgsyerrgdc,87661,3.003866e+18,ACTION_EVM


In [13]:
# all merge events
sql = 'SELECT * FROM merged_event'
merge_event = pd.read_sql(sql, con)

merge_event['claimed_coins'] = merge_event['claimed_coins'].str.replace('aevmos', '').astype(float)
merge_event['fund_community_pool_coins'] = merge_event['fund_community_pool_coins'].str.replace('aevmos', '').astype(float)
merge_event['height'] = merge_event['height'].astype(int)
merge_event['sender_genesis_claim_record'] = merge_event['sender_genesis_claim_record'].astype(float)

merge_event.sample(5)


,id,recipient,sender,height,claimed_coins,fund_community_pool_coins,sender_evmos_prefix,sender_genesis_claim_record
84,85,evmos1h6cxj9hrenfdp5mdqylcxwez05zhwzduprffqj,osmo1l4v8yx7ptzjl0sy5qh9r6me5dlsvrqqau4whkp,274315,1.767573e+20,4.943965e+19,evmos1l4v8yx7ptzjl0sy5qh9r6me5dlsvrqqak0vf6m,1.835300e+20
178,179,evmos156r2mhuycy67q7x8eytm5g9j6znnwd63x6g2pk,osmo1c0l29gvpj7n7wq4mmcrszvn50yt723mdcytfsa,274645,3.076797e+20,8.625474e+19,evmos1c0l29gvpj7n7wq4mmcrszvn50yt723mdj7fhu8,1.835300e+20
1007,1008,evmos1sxk8gsf48avu8p47xm96aa0sctymfhd6eehnpq,osmo1q6mgkye350pq2ykrcu2467xrqn0jspd5gnm969,4880520,1.047311e+19,1.414160e+20,evmos1q6mgkye350pq2ykrcu2467xrqn0jspd5zfemkl,5.752304e+20
496,497,evmos1cznw76m20m8gtea5rmecdxut8lwsd8ej4d2nsr,osmo1m367dtm6ucqs3nw7xlrajgfah270p9p45tkks3,1123680,3.311706e+20,1.653115e+19,evmos1m367dtm6ucqs3nw7xlrajgfah270p9p4735gut,3.357432e+20
667,668,evmos1fkcelrmthq2ykx4xndgm6jtxdzsa2q9evslnj2,osmo18dzfphl03a895hag8wyx97sepuly67esrdy5g8,2796612,8.748742e+19,1.787745e+19,evmos18dzfphl03a895hag8wyx97sepuly67esfhx2ya,4.073588e+20


In [52]:
# Claim Events Decay loss

count_progress = 0
output = []
for index, row in claim_event.iterrows():
    claimable_at_event = 0
    if count_progress == 1000:
        print('processed ', index)
        count_progress = 0
    # find in dataframe row with COLUMN = VALUE
    decay_row = decay_amount.loc[decay_amount['sender'] == row['sender']]
    initial_claimable_amount = decay_row['initial_claimable_amount'].values[0]
    if initial_claimable_amount.is_integer():
        claimable_at_event = initial_claimable_amount
    # find sender in merged_event and check if was not found
    merge_row = merge_event.loc[merge_event['recipient'] == row['sender']]
    # check if we should increase the claimable amount at that block height
    if not merge_row.empty and merge_row['height'].values[0] < row['height']:
        # TODO maybe I should check if the amount received at this height for the sender was good.
        # increasae claimable_at_event
        claimable_at_event = claimable_at_event + merge_row['sender_genesis_claim_record'].values[0]

    # calculate lost amount
    amount_to_claimed = claimable_at_event / 4
    lost_amount = amount_to_claimed - row['amount']

    output.append((row['id'], row['sender'], row['claim_action'], row['height'], initial_claimable_amount, claimable_at_event, row['amount'], lost_amount))
    count_progress = count_progress + 1
result = pd.DataFrame(output, columns=['id', 'sender','event_type', 'height', 'initial_claimable_amount', 'claimable_at_event', 'amount_claimed', 'lost_amount'])
result.head()


processed  1000
processed  2000
processed  3000
processed  4000
processed  5000
processed  6000
processed  7000
processed  8000
processed  9000
processed  10000
processed  11000
processed  12000
processed  13000
processed  14000
processed  15000
processed  16000
processed  17000
processed  18000
processed  19000
processed  20000
processed  21000
processed  22000
processed  23000
processed  24000
processed  25000
processed  26000
processed  27000
processed  28000
processed  29000
processed  30000
processed  31000
processed  32000
processed  33000
processed  34000
processed  35000
processed  36000
processed  37000
processed  38000
processed  39000
processed  40000
processed  41000
processed  42000
processed  43000
processed  44000
processed  45000
processed  46000
processed  47000
processed  48000
processed  49000
processed  50000
processed  51000
processed  52000
processed  53000
processed  54000
processed  55000
processed  56000
processed  57000
processed  58000
processed  59000
proces

,id,sender,event_type,initial_claimable_amount,claimable_at_event,amount_claimed,lost_amount
0,1,evmos18xszt8l8slmm873l2v5w20v5tsh4l7en4d0jex,ACTION_DELEGATE,9.323842e+19,9.323842e+19,2.330960e+19,0.0
1,2,evmos1v80jrjdzrc5zt3q5c8zwkawm0kqx48z84yq0kc,ACTION_VOTE,7.050327e+18,7.050327e+18,1.762582e+18,0.0
2,3,evmos1p3a4h405w5vh4t0q64jsyvm4k8tyx9stujj75x,ACTION_DELEGATE,2.353460e+19,2.353460e+19,5.883650e+18,0.0
3,4,evmos1rgf75fc9chvc2x8p4yaad5llxqzjhfd378z0zt,ACTION_VOTE,4.009982e+19,4.009982e+19,1.002496e+19,0.0
4,5,evmos1az3xkmtpwdhjuwpg8llm2xh7p0ea8f4pjq4v4r,ACTION_DELEGATE,2.170491e+19,2.170491e+19,5.426226e+18,0.0


In [ ]:
result.count()

id                          520043
sender                      520043
event_type                  520043
initial_claimable_amount    472858
claimable_at_event          519989
amount_claimed              520043
lost_amount                 519989
dtype: int64

In [59]:
# generate new 'height' column in dataframe with the height of the event in claims_event dataframe by using the id 
result['height'] = result['id'].apply(lambda x: claim_event.loc[claim_event['id'] == x]['height'].values[0])

In [62]:
# height as int
result['height'] = result['height'].astype(int)
result.sample(5)

,id,sender,event_type,initial_claimable_amount,claimable_at_event,amount_claimed,lost_amount,height
208158,208159,evmos1lmkla8p2ew2fa7qtp7n3fc5z6e4a9724ggyfrs,ACTION_DELEGATE,1.538183e+20,1.538183e+20,3.845458e+19,0.0,192558
404402,404403,evmos19k2ggec2n8n5ynxfqjrj4uhelh0cz7lnlpwyu0,ACTION_EVM,7.050327e+18,7.050327e+18,1.762582e+18,0.0,768292
182425,182426,evmos1l8avr4ecn4j533hcz37jg47ge7rft2lcyen3vk,ACTION_EVM,7.050327e+18,7.050327e+18,1.762582e+18,0.0,170247
472771,472772,evmos1r8jfztdrxupy6a2tycvz43ch0a7nu2j94c69ew,ACTION_IBC_TRANSFER,9.728821e+19,9.728821e+19,2.432205e+19,0.0,1722644
262692,262693,evmos158cta2w6mkm6t2wels2htlnwn4tulehh74h9tn,ACTION_IBC_TRANSFER,1.099643e+20,1.099643e+20,2.749107e+19,0.0,367233


In [65]:
# create new column 'expected_claim' which equals to that row claimable_at_event / 4
result['expected_claim'] = result['claimable_at_event'] / 4

In [67]:
# return the row with min height with lost_amount > 0
result.loc[result['lost_amount'] > 0].sort_values(by=['height']).head(5)

,id,sender,event_type,initial_claimable_amount,claimable_at_event,amount_claimed,lost_amount,height,expected_claim
235302,235303,evmos1al2j48j9fl4e4k8defvgc75hq0t8ehlhupl36d,ACTION_EVM,3.226298e+19,3.226298e+19,6.387250e+18,1.678495e+18,265410,8.065744e+18
235303,235304,evmos14v4n0umtecu38ca67s04u5xnajnghglk4fs23a,ACTION_DELEGATE,4.277243e+19,4.277243e+19,8.467625e+18,2.225482e+18,265426,1.069311e+19
235304,235305,evmos1545k52p32uskfp3aar7sewd6sc3spe9kgyecnw,ACTION_VOTE,1.473063e+19,1.473063e+19,2.916138e+18,7.665203e+17,265442,3.682658e+18
235305,235306,evmos1t8ct8gyv7qpnmsyk45p72eq4vnemat23fpt5sx,ACTION_DELEGATE,7.050327e+18,7.050327e+18,1.395710e+18,3.668714e+17,265443,1.762582e+18
235306,235307,evmos1ktel9ec72nd6addnfvdv2v52zjyqkvcn6f9wsh,ACTION_EVM,7.050327e+18,7.050327e+18,1.395692e+18,3.668893e+17,265452,1.762582e+18


In [68]:
# return the row with max height with lost_amount > 0
result.loc[result['lost_amount'] > 0].sort_values(by=['height'], ascending=False).head(5)

,id,sender,event_type,initial_claimable_amount,claimable_at_event,amount_claimed,lost_amount,height,expected_claim
520042,520043,evmos1pu8sjzk8wwhp3272z948uajfdz0kkjf6l955jd,ACTION_EVM,7.050327e+18,7.050327e+18,5.019794e+14,1.762080e+18,5073387,1.762582e+18
520041,520042,evmos1a66kkjer33wyzhpy7dw839ltk78f4ytkp3upm4,ACTION_EVM,7.050327e+18,7.050327e+18,1.223862e+15,1.761358e+18,5072234,1.762582e+18
520032,520033,evmos1ama6mf2nn7dw9jruz9gumvs9wwvrsv6z4pcc9h,ACTION_IBC_TRANSFER,7.645915e+19,7.645915e+19,1.809251e+16,1.909670e+19,5071523,1.911479e+19
520026,520027,evmos19vra3y4cg8g7aymfv2gu242h4kcuytl2fufu25,ACTION_IBC_TRANSFER,4.485653e+19,4.485653e+19,1.161362e+16,1.120252e+19,5071272,1.121413e+19
520025,520026,evmos1kd6cpfn096t8uvf0pveatvseyarfrz2lccdgkj,ACTION_VOTE,NaN,6.871449e+20,2.307809e+17,1.715554e+20,5071204,1.717862e+20


In [70]:
result.count()

id                          520043
sender                      520043
event_type                  520043
initial_claimable_amount    472858
claimable_at_event          519989
amount_claimed              520043
lost_amount                 519989
height                      520043
expected_claim              519989
dtype: int64

In [69]:
# update csv file
result.to_csv('claim_event_decay_loss.csv', index=False)

In [72]:
# return the sum of all lost_amount in rows with lost_amount > 0 divided by 10^18
result.loc[result['lost_amount'] > 0]['lost_amount'].sum() / 1000000000000000000

158971.30615962716

In [76]:
# get count of rows where lost_amount < 0
result.loc[result['lost_amount'] < 0].count("lost_amount")

id                          51325
sender                      51325
event_type                  51325
initial_claimable_amount     4170
claimable_at_event          51325
amount_claimed              51325
lost_amount                 51325
height                      51325
expected_claim              51325
dtype: int64

In [77]:
# generate csv file from result with accounts where lost_amount > 0 sorted by address
result.loc[result['lost_amount'] > 0].to_csv('claim_event_decay_loss_with_loss.csv', index=False)
result.loc[result['lost_amount'] > 0].to_csv('claim_event_decay_loss_with_loss.csv', index=False)